# CodeNet Dataset

In this notebook I intend to make it very easy to visualize all the necessary information about a specific problem. The base dataset I will be using in this notebook is [CodeNet](https://github.com/IBM/Project_CodeNet) which is a large collection of source files and problem descriptions with metadata. The solutions are written in multiple programming languages (55+ according to the paper) and each problem has multiple submissions. Most of the submissions are written in the six most common languages (C++, Python, Java, C, Ruby, C#). As expected most of the solutions are in C++. One interesting aspect of the dataset is that it includes failed submissions, with various status codes such as Compilation Errors, Runtime Errors, Time Limit Exceeded, Memory Limit Exceeded, etc. This will prove useful since we are looking into bug detection in source code files.

Make sure you run `source spt.profile` to create the environment for the tokenizer to work. Also you have to compile the tokenizer from CodeNet.

## Table of Contents
1. [Imports](#Imports)
1. [Download CodeNet](#Download-CodeNet)
1. [Missing Values](#Missing-Values)

## Imports

In [1]:
import myparser
import codenet

import pandas as pd

pd.set_option('max_columns', None)

codenet.P = 4

## Download CodeNet

The next code cell will download the CodeNet dataset from it's original repository (the archive has around 80GB). If you already have the dataset change the input_path variable to point to the root of the dataset, otherwise the notebook will download it in the ../input/ directory.

In [3]:
codenet.download_data()

dataset root dir found


## Missing Values

The dataset also includes a description file for most of the problems. We can see which problems have or don't have a description associated. The description file can be useful to predict what the problem topic is about, graphs, dp, greedy, etc.

In the case of missing input files, I think it is also better to just drop the submissions, most of the description files are written in Chinese and we cannot really extract any useful information from them. Since there are so few files with no input we can drop them. By looking in the description files there are like 2 problems with no input from the stdin.

To conclude the missing values section, 54/56 of the missing names in the problems list are due to missing description files 1/56 is just a href which links to a 404 web page and the last one is a test problem, the later 2 problems having no submissions anyway. I think it is a fair decision to drop these samples as they are not useful. There will be 130 remaining problems with no input/output samples and 128 of them have description files in Chinese which makes it harder to extract samples, and 2 of them only require printing of values (similar to problem p00000). In this case I also think that it is ok to drop those 2 problems that don't need input alongside the rest of problems that have no input examples extracted, because we don't have to remember that there is one or two problems that can cause some bugs later on.

In [4]:
codenet.clean_problem_list().to_csv(codenet.problem_list_clean_v2_path)

Cleaning ../input/Project_CodeNet/metadata/problem_list.csv


In [2]:
problem_list_df = pd.read_csv(codenet.problem_list_clean_v2_path, index_col="id")
problem_ids = problem_list_df.index.unique()

print(f"We have {len(problem_list_df)} problems")
print('The distribution of the datasets is')
print(problem_list_df['dataset'].value_counts(normalize=True))
display(problem_list_df.head())
display(problem_list_df.isna().sum())

We have 3867 problems
The distribution of the datasets is
AIZU       0.613654
AtCoder    0.386346
Name: dataset, dtype: float64


,name,dataset,time_limit,memory_limit,rating,tags,complexity
id,,,,,,,
p00001,List of Top 3 Hills,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00002,Digit Number,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00003,Is it a Right Triangle?,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00004,Simultaneous Equation,AIZU,1000.0,131072.0,NaN,NaN,NaN
p00005,GCD and LCM,AIZU,1000.0,131072.0,NaN,NaN,NaN


name               0
dataset            0
time_limit         0
memory_limit       0
rating          3867
tags            3867
complexity      3867
dtype: int64

## Generate Source Code Pairs
- for each problem:
- group solutions by user and select the solutions that are consecutive and of the form (Error, Accepted) and sort by the submission date; heuristic to search a smaller space, since a user might submit a correct solution after a wrong one
- build a df from this list and save it

Here we want to generate submission pairs that will allow us to find code fixes within the dataset. With this information we will be able to find the instructions that have to be modified, either deleted, inserted or changed, so that the old code starts to work. You would imagine that solutions that were submitted successively should be similar in terms of content and only have a few changes between each other. The small mistakes can be patched by knowing the accepted submission in that chain and give us the errors produced, and thus removed by the correct instruction. These are the submissions that we are interested in and look into finding patterns and further cleaning those that will not prove helpful.

In [ ]:
codenet.generate_pairs_v2(problem_list_df).to_csv(generated_pairs_v2_path, index=False)

In [3]:
generated_pairs_df = pd.read_csv(codenet.generated_pairs_v2_path)

display(generated_pairs_df)
display(generated_pairs_df.info())
display(generated_pairs_df.language.value_counts())
display(generated_pairs_df.original_status.value_counts())

print(f"We are left with {len(generated_pairs_df)} submissions in total")

,original_id,changed_id,original_status,problem_id,language,filename_ext
0,s000016565,s604436209,Runtime Error,p03106,Python,py
1,s000023530,s834210063,Runtime Error,p02684,Python,py
2,s000041036,s454210115,Runtime Error,p02584,Python,py
3,s000041460,s952454189,Time Limit Exceeded,p02658,Python,py
4,s000054326,s226572665,Time Limit Exceeded,p02701,Python,py
...,...,...,...,...,...,...
54587,s999855642,s185899176,Runtime Error,p02645,Python,py
54588,s999876744,s590378402,Time Limit Exceeded,p02642,Python,py
54589,s999891212,s835176880,Runtime Error,p03103,Python,py
54590,s999921259,s180756175,Time Limit Exceeded,p02713,Python,py


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54592 entries, 0 to 54591
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   original_id      54592 non-null  object
 1   changed_id       54592 non-null  object
 2   original_status  54592 non-null  object
 3   problem_id       54592 non-null  object
 4   language         54592 non-null  object
 5   filename_ext     54592 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


None

Python    54592
Name: language, dtype: int64

Runtime Error             34126
Time Limit Exceeded       18385
WA: Presentation Error     1938
Memory Limit Exceeded       134
Output Limit Exceeded         8
Judge Not Available           1
Name: original_status, dtype: int64

We are left with 54592 submissions in total


## Tokenize Source Code Pairs

next